In [ ]:
import pandas as pd
import numpy as np

# Importing Dataset

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Datasets/fashion-mnist_train.csv')

# Splitting and Scalling

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop('label', axis=1)
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_x = X_train/255.0
test_x = X_test/255.0

# Check for GPU Availability

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))

Tesla T4


# Dataset Class and DataLoader

In [ ]:
from torch.utils.data import Dataset, DataLoader

class CustomDataSet(Dataset):
  def __init__(self, x, y):
    self.x = torch.tensor(x.values, dtype=torch.float32)
    self.y = torch.tensor(y.values, dtype=torch.long)

  def __getitem__(self, i):
    return self.x[i], self.y[i]

  def __len__(self):
    return len(self.x)

train_data = CustomDataSet(train_x, y_train)
test_data = CustomDataSet(test_x, y_test)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False, pin_memory=True)

# Defining the Model

In [ ]:
import torch.nn as nn

class MyModel(nn.Module):
  def __init__(self, num_features):
    super().__init__()

    self.layer1 = nn.Sequential(
        nn.Linear(num_features, 128),
        nn.BatchNorm1d(128), # Normalised the neurone
        nn.ReLU()
    )

    self.layer2 = nn.Sequential(
        nn.Dropout(0.3), # Drop out some neurone
        nn.Linear(128, 64),
        nn.BatchNorm1d(64), # Normalised the neurone
        nn.ReLU()
    )

    self.output_layer = nn.Linear(64, 10)

  def forward(self, x):
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.output_layer(x)
    return x

In [ ]:
# Defining Parameter
learning_rate = 0.01
epochs = 100

In [ ]:
# Initialising Model
model = MyModel(train_x.shape[1]).to(device)

# loss Function
loss_fn = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-4) # penalise the loss function

# Model Pipeline

In [ ]:
for epoch in range(epochs):

  total_epoch_loss = 0
  for batch_features, batch_labels in train_loader:
    batch_features = batch_features.to(device)
    batch_labels = batch_labels.to(device)

    # Forward Pass
    output = model(batch_features)
    loss = loss_fn(output, batch_labels)

    # Backward Pass
    optimizer.zero_grad()
    loss.backward()

    # Update Gradient
    optimizer.step()

    total_epoch_loss += loss.item()

  print(f'Epoch: {epoch+1}, Loss: {total_epoch_loss/len(train_loader)}')

Epoch: 1, Loss: 0.8788576284249624
Epoch: 2, Loss: 0.5430560364524524
Epoch: 3, Loss: 0.4787677427728971
Epoch: 4, Loss: 0.443377348780632
Epoch: 5, Loss: 0.4184263183871905
Epoch: 6, Loss: 0.4045162236491839
Epoch: 7, Loss: 0.38858470602830253
Epoch: 8, Loss: 0.3768984512289365
Epoch: 9, Loss: 0.3664283930460612
Epoch: 10, Loss: 0.3570358684659004
Epoch: 11, Loss: 0.3494120441973209
Epoch: 12, Loss: 0.34409622504313786
Epoch: 13, Loss: 0.335622811794281
Epoch: 14, Loss: 0.3286780047218005
Epoch: 15, Loss: 0.3237002332806587
Epoch: 16, Loss: 0.3185934271613757
Epoch: 17, Loss: 0.31147253163655597
Epoch: 18, Loss: 0.3063351659576098
Epoch: 19, Loss: 0.3026061212221781
Epoch: 20, Loss: 0.29922945508360865
Epoch: 21, Loss: 0.2942378022869428
Epoch: 22, Loss: 0.28863375347852704
Epoch: 23, Loss: 0.2859149602552255
Epoch: 24, Loss: 0.28364309947689376
Epoch: 25, Loss: 0.28030620063344636
Epoch: 26, Loss: 0.27542208358645437
Epoch: 27, Loss: 0.27095264651378
Epoch: 28, Loss: 0.26876924932003

# Model Evaluation

In [ ]:
model.eval()

total_correct = 0
total_samples = 0

with torch.no_grad():
  for batch_features, batch_labels in test_loader:
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    output = model(batch_features)
    _, predicted = torch.max(output, 1)

    total_samples += batch_labels.size(0)
    total_correct += (predicted == batch_labels).sum().item()

accuracy = total_correct / total_samples
print(f'Accuracy: {accuracy}')

Accuracy: 0.8939166666666667


In [ ]:
model.eval()

total_correct = 0
total_samples = 0

with torch.no_grad():
  for batch_features, batch_labels in train_loader:
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    output = model(batch_features)
    _, predicted = torch.max(output, 1)

    total_samples += batch_labels.size(0)
    total_correct += (predicted == batch_labels).sum().item()

accuracy = total_correct / total_samples
print(f'Accuracy: {accuracy}')

Accuracy: 0.9703958333333333
